In [1]:
from PIL import Image
import requests
import json
from PIL import Image
import torch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import itertools
from tqdm.autonotebook import tqdm
import albumentations as A
import matplotlib.pyplot as plt
from dataset.data_module import DataModule
import torch
from torch import nn
import torch.nn.functional as F
import timm
from PIL import Image
import requests
from transformers import (
    VisionTextDualEncoderModel,
    VisionTextDualEncoderProcessor,
    AutoImageProcessor,
    AutoTokenizer,
)
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

/home/wyh21/anaconda3/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-03-22 17:01:17.732783: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 17:01:17.779238: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 17:01:18.633430: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#image_processor = AutoImageProcessor.from_pretrained("microsoft/swin-base-patch4-window7-224")
bert_path='/data/wyh21/huggingface/models--emilyalsentzer--Bio_ClinicalBERT'
tokenizer = AutoTokenizer.from_pretrained(bert_path)
model = VisionTextDualEncoderModel.from_vision_text_pretrained(
    "microsoft/swin-base-patch4-window7-224", bert_path
)
image_processor = AutoImageProcessor.from_pretrained("microsoft/swin-base-patch4-window7-224")
processor = VisionTextDualEncoderProcessor(image_processor, tokenizer)

/home/wyh21/anaconda3/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
The projection layer and logit scale weights `['visual_projection.weight', 'text_projection.weight', 'logit_scale']` are newly initialized. You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [7]:
class Args():
    batch_size=108
    val_batch_size=108
    num_workers=12
    prefetch_factor=4
    annotation=r'/data/wyh21/physionet.org/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic_annotation_all.json'
    vision_model='microsoft/swin-base-patch4-window7-224'
    base_dir=r'/data/wyh21/physionet.org/physionet.org/files/mimic-cxr-jpg/2.0.0/files'
    #  num_nodes=args.num_nodes,
    #     strategy=args.strategy,
    #     accelerator=args.accelerator,
    #     precision=args.precision,
    #     val_check_interval = args.val_check_interval,
    #     limit_val_batches = args.limit_val_batches,
    #     max_epochs = args.max_epochs,
    #     num_sanity_val_steps = args.num_sanity_val_steps,
    #     accumulate_grad_batches=args.accumulate_grad_batches,
    num_nodes=1,
    strategy="ddp",
    accelerator="gpu",
    precision='bf16-mixed',
    val_check_interval = 1.0,
    limit_val_batches = 1.0,
    max_epochs = 50,
    num_sanity_val_steps = 0,
    accumulate_grad_batches=1

args=Args()
dm = DataModule(args)

12


In [8]:
import lightning.pytorch as pl
trainer = pl.Trainer(
       # devices=args.devices,
        #devices=[0,2,3,4], 
        devices=[0],
        num_nodes=args.num_nodes,
        strategy=args.strategy,
        accelerator=args.accelerator,
        precision=args.precision,
        val_check_interval = args.val_check_interval,
        limit_val_batches = args.limit_val_batches,
        max_epochs = args.max_epochs,
        num_sanity_val_steps = args.num_sanity_val_steps,
        accumulate_grad_batches=args.accumulate_grad_batches,
    )


ValueError: You selected an invalid strategy name: `strategy=('ddp',)`. It must be either a string or an instance of `lightning.pytorch.strategies.Strategy`. Example choices: auto, ddp, ddp_spawn, deepspeed, ... Find a complete list of options in our documentation at https://lightning.ai

In [ ]:
trainer.fit(model, datamodule=dm)